# FLDAS Explorer Dashboard
Modified: Jun 17, 2019

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import datetime as dt
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
import intake,param
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

from tqdm import tqdm, trange
import ipywidgets as iw

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr
import xarray.ufuncs as xu

from holoviews import opts
from holoviews.util import Dynamic
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize, aggregate

from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import panel as pn

import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf


hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")    

In [ ]:
#import utils
import hv_utils as  hvu

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

## Set default holoviews style options

In [ ]:
%opts Image [colorbar=True, tools=['hover'], active_tools=['wheel_zoom']] Curve [tools=['hover']]

In [ ]:
opts.defaults(
    opts.WMTS(active_tools=['wheel_zoom']),
    opts.Image(active_tools=['wheel_zoom'], tools=['hover'], colorbar=True),
    opts.Curve(active_tools=['wheel_zoom'], tools=['hover']),
    opts.Scatter(active_tools=['wheel_zoom'], tools=['hover']),
    opts.HLine(active_tools=['wheel_zoom'], tools=['hover']),

    opts.RGB(active_tools=['wheel_zoom']),
    opts.Overlay(active_tools=['wheel_zoom']),
)


In [ ]:
H,W = 500,500

---
## Load Datasets

In [ ]:
# Southern Africa Dataset
data_dir = Path.home()/'data'
fpath_sa = str(
    data_dir/'mint/FLDAS/FLDAS_NOAH01_A_SA_D.001/2019/04/FLDAS_NOAH01_A_SA_D.A201904*.001.nc'
)
fpath_ea = str(
    data_dir/'mint/FLDAS/FLDAS_NOAH01_A_EA_D.001/2019/04/FLDAS_NOAH01_A_EA_D.A201904*.001.nc'
)
ds_sa = xr.open_mfdataset(fpath_sa)
ds_sa = ds_sa.drop_dims('bnds')

ds_ea = xr.open_mfdataset(fpath_ea)
ds_ea = ds_ea.drop_dims('bnds')

         
# print(ds_ea)
# print(ds_sa)

In [ ]:
xrd_ea = ds_ea.persist()
xrd_sa = ds_sa.persist()

In [ ]:
# data variable list
varnames_ea = list(ds_ea.data_vars.keys())
varnames_sa = list(ds_sa.data_vars.keys())
varnames = varnames_ea
varname = varnames[3]
print(varname)

# create holoviews dataset containers 
kdims = ['X','Y','time']
hvd_ea = hv.Dataset(xrd_ea, kdims)
hvd_sa = hv.Dataset(xrd_sa, kdims)


In [ ]:
# colormaps
## discretize it conveniently using holoview's "color_level" option
t_fixed = '2019-04-05'
varname = varnames[5] 
print("Selecting a datavariable at a fixed time point: ", t_fixed, varname)

# timg_ea = hvd_ea.select(time=t_fixed).to(gv.Image, kdims=['X', 'Y'], vdims=varname) #this returns a holomap, not a hv.Image object
# To construct an hv.Image object, we need to pass in the xr.DataArray (ie. one value variable)
print(xrd_ea[varname].isel(time=3) )
timg_ea = gv.Image(xrd_ea[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
timg_sa = gv.Image(xrd_sa[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
# print(timg_sa)
# gv.tile_sources.Wikipedia * timg_ea.opts(alpha=0.5,width=W_IMG, height=H_IMG) #+ timg_sa.opts(width=W_IMG, height=H_IMG)

## Basemap tile

We need to handle the projection from latlon to web mercator (which is what the hv.tiles expect).


In [ ]:
wmts_url = 'https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png'
basemap = gv.tile_sources.EsriImagery

---
## Add Callbacks 

### LatLon Tab selector stream

Fetch the lat,lon location from the mouse location

In [ ]:
from holoviews.streams import Tap, Selection1D, PointerXY, RangeXY

In [ ]:
# Defne time sereis scatter point selector stream
# tscatter_selection = Selection1D(source=tseries.Scatter.I)

In [ ]:
def cb_rangexy(x_range, y_range):
    lbrt = hvu.ranges2lbrt(x_range, y_range)
    print(f'x_range: {x_range}')
    print(f'y_range: {y_range}')
    print(f'lbrt: ', lbrt)
    
    df = pd.DataFrame( [lbrt], columns='min_x min_y max_x max_y'.split() )
    return hv.Table( df)


---
## Putting the streams together


## FLDASExplorer with panel
Modified: Jun 13, 2019

In [ ]:
# Set extra style opts (in addition to default from above)
W_IMG = 400; H_IMG = 400
W_PLOT = 800; H_PLOT = 300

In [ ]:
scatter_opts = dict(width=W_PLOT, height=H_PLOT,
                    tools=['hover', 'tap'], 
                    framewise = True)
curve_opts = dict(width=W_PLOT, height=H_PLOT,
                  framewise=True)
img_opts = dict(width=W_IMG, height=H_IMG,
                axiswise=True, 
                framewise=False,
                tools=['hover', 'tap'],
                colorbar=True
               )
wmts_opts = dict(width=W_IMG, height=H_IMG)

tbl_opts = dict(width = W_PLOT)

# datashader opts
ds_opts = dict(width=W_IMG, height=H_IMG,
#             x_sampling=0.5, 
#             y_sampling=0.5,
            )

In [ ]:
import datetime as dt


trange = list(map(pd.Timestamp, hvd_ea.range('time')))
basemap.opts(**wmts_opts)

class FLDASExplorer(param.Parameterized):

    ################################################################################
    ## Parameters
    ################################################################################
    region = param.ObjectSelector(default='EA', objects=['EA', 'SA'])
    varname = param.ObjectSelector(default=varnames[0],objects=varnames)
    time = param.Date(trange[14], bounds=trange)
#     time_slider= param.Date(trange[0], bounds=trange)


    ################################################################################
    ## Initialize instance
    ################################################################################
    def __init__(self):
        super().__init__()
        self._init_streams()
        self._init_debug_box()
        self._set_dyn_main()
        self._set_dyn_funcs()
        
        
    ################################################################################
    ## Initialize Link streams inbetween static components
    ################################################################################
    @param.depends('region', 'varname', 'time', watch=True)
    def _set_dyn_main(self):
        print("Setting main plot dynamically...")
        print('\tTime: ', self.time, type(self.time))
        dmap_img = hv.DynamicMap(self.get_main_img)
        dyn_img = datashade(Dynamic(dmap_img),**ds_opts)

        self.dyn_main = basemap * dyn_img
    
        # must relink the streams 
        self.dyn_img = dyn_img
        self.tap_stream.source = self.dyn_img
    
    def _set_dyn_funcs(self):
        # todo: add set_dyn_main
        
        self.dyn_tseries = hv.DynamicMap(self.cb_tseries, streams=[self.tap_stream] )
        self.dyn_tstats = hv.DynamicMap(self.cb_tstats, streams=[self.tap_stream])
        self.dyn_second_plot = basemap* datashade(hv.DynamicMap(self.cb_second_plot, streams=[self.sel_stream]),
                              **ds_opts)
        
        
    def _init_streams(self):
        self.tap_stream = Tap(name='tap_latlon', x=35.0, y=0.0)#, source=self.main_plot)
        self.sel_stream = Selection1D()
        self.dummy = Tap(name='dummy', x=0, y=0)
        self.streams = [self.tap_stream, self.sel_stream]
        print("Created initial tap and selection1d stream instances...")

    def _init_debug_box(self):
        print("Initialing a debug box...")
        self.debug_box = hv.Div("""
        <h1 style='color:blue;border:1px solid blue'>Debug Box</h1>
        """)
        self.debug_dmap = hv.DynamicMap(self.get_updated_debug_box, 
                                       streams=self.streams)
        print('Initiated debug_dmap with current streams to debug_box')
        
    def get_updated_debug_box(self, *args, **kwargs):
        hvu.append_to_div(self.debug_box, str(args))
        hvu.append_to_div(self.debug_box, str(kwargs))
        return self.debug_box
    
    
    ################################################################################
    ## Dynamic Building Blocks (hv.DynamicMaps)
    ################################################################################        
    @param.depends('region', 'varname', 'time')#, watch=True)
    def get_main_img(self):
        print("Getting a new main img...")
        print(f'\tTime: {self.time}, Region: {self.region}')
        xrd = xrd_ea if self.region == 'EA' else xrd_sa
        img = hvu.get_img(xrd, self.varname, self.time)
        # debug
        
        return img.relabel(self.varname).opts(**img_opts)

        
    ################################################################################
    ## Link streams Callbacks
    ################################################################################
    @param.depends('region', 'varname')
    def cb_tseries(self, x, y, method='nearest'):
        """
        Affects:
        - Until the first time this callback  called by the instance, there is no
        self.tscatter attribute 
        - Everytime it is called, self.tscatter is updated to the new scatter plot 
        object that resides in the caller hv.DynamicMap.plot instance
            - The reason to maintain which self.tscatter we use is to 
        """
        print(f'cb_tseries is called. x,y: {x,y, self.region,self.varname}"')
        xrd = xrd_ea if self.region == 'EA' else xrd_sa  
        tseries = xrd[self.varname].sel(X=x, Y=y, method=method)
        tseries_label = f"Time Series for {self.varname} at Lon,Lat = ({x:.2f},{y:.2f}) "

        # Time series as Scatter and Curve
        print('\tself.scatter is updated')
        self.tscatter = hv.Scatter(tseries, label=self.varname)
        curve = hv.Curve(tseries)

        # Important! Connect the self.sel_stream to this new tscatter
        print("\tself.sel_stream's source is changed to the new self.tscatter object")
        self.sel_stream.source = self.tscatter

        # Add HLine at mean over time
        mean = hvu.extract_item(tseries.mean())
        mean_line = hv.HLine(mean, label='tseries_mean')
        t_midpoint = pd.Timestamp(hvu.extract_item(tseries.coords['time'][len(tseries)//2]))
        mean_label = hv.Labels([(t_midpoint, mean, f'mean: {mean:.3f}')])

        # Put together to an Overlay
        overlay = (
            curve.opts(alpha=0.5, line_width=2, **curve_opts) 
            * self.tscatter.opts(padding=0.2, size=10, **scatter_opts) 
            * mean_line.opts(color='black', alpha=0.5, line_width=1)
            * mean_label.opts(text_font_size='8pt',text_alpha=0.5)
        ) 
        overlay = overlay.relabel(tseries_label)
#         self.overlay = overlay

        return overlay #vs. self.overlay
    
    @param.depends('region', 'varname')
    def cb_tstats(self, x, y, 
                  method='nearest',show_as_timestamp=True, decimals=3):
        print(f"cb_tstats is called: {x, y, self.region, self.varname}")
        xrd = xrd_ea if self.region == 'EA' else xrd_sa  
        tseries = xrd[self.varname].sel(X=x, Y=y, method=method)
        df = hvu.get_stats(tseries, 
                           show_as_timestamp=show_as_timestamp, decimals=decimals)

        # Add metadata on selected latlon point
    #     df['point_idx'] = index[0]#
        df['lat'] = y
        df['lon'] = x

        cols = df.columns.to_list()
        cols = cols[-2:] + cols[:-2]
        df = df[cols]
        label = f"Time Series Stats for {self.varname} at Lon,Lat = ({x:.2f},{y:.2f}) "
        print("\tUpdated self.tstats_table")
        self.tstats_table = hv.Table(df, label=label).opts(**tbl_opts)
        return self.tstats_table
    
    @param.depends('region', 'varname')
    def cb_second_plot(self, index):
        print('Callback from the scatter plot is called. Selected indices: ', index)
        if not index:
            index = [0] #todo: current tindex
        tidx = index[0]
        
        xrd = xrd_ea if self.region == 'EA' else xrd_sa 
        img = hvu.get_img(xrd, self.varname, tidx)
        label = f"{self.varname} on {self.time.strftime('%Y/%m/%d')}"
        return img
#         dmap_second = hv.DynamicMap(img)
#         dyn_second = datashade(Dynamic(dmap_second), **dopts)
# #         pdb.set_trace()
#         return  basemap * dyn_second
    

    ################################################################################
    ## Build views
    ################################################################################
    @param.depends('region', 'varname', 'time')
    def view(self):
        row1 = (self.dyn_main + self.dyn_second_plot).opts(
            opts.Image(**img_opts))
        row2 = self.dyn_tseries.opts(
            opts.Scatter(**scatter_opts),
            opts.Curve(**curve_opts))
        row3 = self.dyn_tstats.opts(opts.Table(**tbl_opts) )

        return pn.Column(row1, row2, row3)

In [ ]:
explorer = FLDASExplorer()
widgets = pn.Param(explorer.param, widgets={
#     'time_slider': pn.widgets.DateSlider,
    'time': pn.widgets.DatePicker
})
# widgets

In [ ]:
pn.Column(explorer.param, explorer.view)

In [ ]:
# row1 = pn.Row( widgets, explorer.dyn_main, explorer.dyn_second_plot
# #               pn.Column (explorer.dyn_tstats, explorer.dyn_tseries )
#              )
# row2 = pn.Row(pn.Spacer(width=300), explorer.dyn_tseries)
# pn.Column(row1, row2)#.servable()


In [ ]:
# app = pn.Row( explorer.param, explorer.view)
# app = pn.Column( explorer.param, 
#                 pn.panel(explorer.build_main_plot),
#                 pn.panel( explorer.build_tseries_plot), 
#                 pn.panel( explorer.build_second_plot),
# #                 pn.panel( explorer.build_tstats_table),
#                )

In [ ]:
# app.servable()


---
## Arrange widgets 
Modified: Jun 19, 2019

In [ ]:
title = """#MINT Interactive Visualization for Spatiotemporal Data"""
instruction = """
This is a demo of an interactive visualization for a large set of spatiotemporal data. 
Here we use the FLDAS NOAH01 daily data in Southern Africa (SA) and Eastern Africa (EA) for the month of April in 2019.
This demo supports a **correct** visualization of a **large** dataset without falling into common visualization [mistakes](#) 
such as overplotting and deceptive colormapping via [Datashader]() and [Colorcet]().


## Instructions

1. Dataset selection

    - dropdown boxes: choose `region`, `variable`, `time`  of interest
    - To specify `time`, a user can either type a specific time to the `Time` input box
    or use the time slider to navigate through multiple time points

2. All maps are responsive to user inputs from the mouse or trackpad.

    - pan or zoom: navigate the map
    - mouse hover or click: get its value or see through multiple time points at a specific (Lat, Lon)
    - drag or box-select: compute zonal statistics in the selected area

3. The time-series plot also supports interactivity

    - pan, zoom, mouse-hover as above
    - click on a point: allows to see the data at the current timepoint over the entire region

## Suggestions?
We would like to know what kind of functionalities users are most interested in.  
Please drop a line for requests, and we'll start working on adding supports for popular requests!

## Questions?
Please feel free to contact haejinso@usc.edu for any questions.
"""

viz_title = """#FLDAS Explorer POC"""
tseries_title = """##Time Series Analysis"""



In [ ]:
W_HEADER = 300 + W_IMG*2
texts = pn.Column( pn.Pane(title, width=W_HEADER),
          pn.Pane(instruction, width=W_HEADER))
viz_title = pn.Pane(viz_title)
tseries_title = pn.Pane(tseries_title)



In [ ]:
# # OLD
# # Viz main panel components
# header = pn.Column(texts,viz_title)
# viz_panel = pn.Column(explorer.view)
# row1 = pn.Row( widgets, explorer.dyn_main, explorer.dyn_second_plot)
# row2 = pn.Row( tseries_title)
# row3 = pn.Row(pn.Spacer(width=300), explorer.dyn_tseries)
# row4 = pn.Row(pn.Spacer(width=300), explorer.dyn_tstats)
# pn.Column(header,row1, row2, row3, row4).servable()

In [ ]:
# main panel components
header = pn.Column(texts,viz_title)
viz_panel = pn.Column(explorer.view)

In [ ]:
row1 = pn.Row(widgets, viz_panel)
pn.Column(header, row1).servable()

In [ ]:
# row1 = pn.Row(widgets) explorer.dyn_main, explorer.dyn_second_plot)
# row2 = pn.Row( tseries_title)
# row3 = pn.Row(pn.Spacer(width=300), explorer.dyn_tseries)
# row4 = pn.Row(pn.Spacer(width=300), explorer.dyn_tstats)
# pn.Column(header,row1, row2, row3, row4).servable()